In [1]:
import pandas as pd

In [2]:
checkout = pd.read_csv("checkout.csv", parse_dates=[1])
checkout.drop_duplicates(["user_id"], inplace=True)
checkout.to_csv("checkout-dedupe.csv", index=False)

Import all the files


In [3]:
visits = pd.read_csv("visits.csv", parse_dates=[1])
cart = pd.read_csv("cart.csv", parse_dates=[1])

checkout = pd.read_csv("checkout.csv", parse_dates=[1])
purchase = pd.read_csv("purchase.csv", parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`


In [4]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart


In [5]:
visits_cart = pd.merge(visits, cart, how="left")
visits_cart.head()

,user_id,visit_time,cart_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT


Step 3: How long is `visits_cart`?


In [6]:
len(visits_cart)

2000

Step 4: How many timestamps are null for `cart_time`?


In [7]:
null_cart_times = visits_cart[visits_cart.cart_time.isnull()]
len(null_cart_times)

1652

Step 5: What percentage only visited?


In [8]:
len(null_cart_times) / len(visits_cart)

0.826

Step 6: What percentage placed a t-shirt in their cart but did not checkout?


In [9]:
cart_checkout = cart.merge(checkout, how="left")
cart_checkout.head()

,user_id,cart_time,checkout_time
0,2be90e7c-9cca-44e0-bcc5-124b945ff168,2017-11-07 20:45:00,2017-11-07 21:14:00
1,4397f73f-1da3-4ab3-91af-762792e25973,2017-05-27 01:35:00,NaT
2,a9db3d4b-0a0a-4398-a55a-ebb2c7adf663,2017-03-04 10:38:00,2017-03-04 11:04:00
3,b594862a-36c5-47d5-b818-6e9512b939b3,2017-09-27 08:22:00,2017-09-27 08:26:00
4,a68a16e2-94f0-4ce8-8ce3-784af0bbb974,2017-07-26 15:48:00,NaT


In [10]:
null_checkout_time = cart_checkout[cart_checkout.checkout_time.isnull()]
len(null_checkout_time) / len(cart_checkout)

0.3505747126436782

Step 7: Merge it all together


In [11]:
# all_data = visits_cart.merge(cart_checkout, how="left").merge(purchase, how="left")
all_data = (
    visits.merge(cart, how="left")
    .merge(checkout, how="left")
    .merge(purchase, how="left")
)
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase


In [12]:
reached_checkout = all_data[~all_data.checkout_time.isnull()]
reached_checkout.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
14,486480e2-98c3-4d51-8f4b-b1c07228ce84,2017-01-27 16:34:00,2017-01-27 16:44:00,2017-01-27 17:10:00,2017-01-27 17:12:00
31,1868e912-b38c-4ef0-8824-5665c42288e5,2017-03-17 11:48:00,2017-03-17 11:56:00,2017-03-17 11:58:00,NaT
48,3ccdaf69-2d30-40de-b083-51372881aedd,2017-01-08 20:21:00,2017-01-08 20:38:00,2017-01-08 20:52:00,2017-01-08 21:02:00
49,3ccdaf69-2d30-40de-b083-51372881aedd,2017-01-08 20:21:00,2017-01-08 20:38:00,2017-01-08 20:52:00,2017-01-08 21:21:00


In [13]:
checkout_not_purchased = reached_checkout[reached_checkout.purchase_time.isnull()]
checkout_not_purchased.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
31,1868e912-b38c-4ef0-8824-5665c42288e5,2017-03-17 11:48:00,2017-03-17 11:56:00,2017-03-17 11:58:00,NaT
54,dc5d8fe9-3321-4c86-b66d-7c343decdc4a,2017-09-13 13:06:00,2017-09-13 13:30:00,2017-09-13 13:26:00,NaT
146,fffc2dfc-fdc1-407a-9c6d-e6507157d1a6,2017-07-03 19:20:00,2017-07-03 19:40:00,2017-07-03 19:51:00,NaT
151,0182a7b9-2099-4476-8f7d-51f2c2f71dff,2017-09-08 16:26:00,2017-09-08 16:34:00,2017-09-08 16:59:00,NaT
161,2faac6a9-974c-49fb-b815-44f3a5fc06e1,2017-03-27 21:51:00,2017-03-27 21:54:00,2017-03-27 22:19:00,NaT


In [14]:
len(checkout_not_purchased) / len(reached_checkout)

0.24550898203592814

Step 9: check each part of the funnel, let's print all 3 of them again


In [15]:
only_visited = len(null_cart_times) / len(visits_cart)
only_cart = len(null_checkout_time) / len(cart_checkout)
only_checkout = len(checkout_not_purchased) / len(reached_checkout)
print(
    f"{round(only_visited * 100, 2)} of users that visit the website didn't add any t-shirt to the cart."
)
print(
    f"{round(only_cart * 100, 2)} of users that added a t-shirt to the cart did't checkout."
)
print(
    f"{round(only_checkout * 100, 2)} of users that checked out did't purchase any t-shirt."
)

82.6 of users that visit the website didn't add any t-shirt to the cart.
35.06 of users that added a t-shirt to the cart did't checkout.
24.55 of users that checked out did't purchase any t-shirt.


_The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page._

Step 10: adding new column


In [16]:
all_data["time_to_purchase"] = all_data.purchase_time - all_data.visit_time

Step 11: examine the results


In [17]:
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT


Step 12: average time to purchase


In [18]:
all_data.time_to_purchase.mean()

Timedelta('0 days 00:43:12.380952380')